# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [29]:
import pandas as pd
import lightgbm as lgb
import re
import time
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from catboost import CatBoostRegressor

In [4]:
data = pd.read_csv('/datasets/autos.csv')

In [5]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [7]:
data.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Приведение названия столбцов к *snake_case*.

In [8]:
def snake_case(name):
    name = re.sub(r'([A-Z]+)([A-Z][a-z])', r'\1_\2', name)
    name = re.sub(r'([a-z\d])([A-Z])', r'\1_\2', name)
    name = re.sub(r'-', '_', name)
    return name.lower()

data.columns = [snake_case(col) for col in data.columns]
data.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')

Удаление неинформативных признаков.

На цену машины как объекта никак не может влиять дата публикации объявления, дата его скачивания, количество картинок, почтовый индекс владельца анкеты и его последний онлайн.

In [9]:
data = data.drop(['date_crawled', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen'], axis=1)

Заполнение пропусков в столбцах 'vehicle_type', 'gearbox', 'fuel_type' наиболее часто встречающимся значением и в столбце 'model' наиболее популярной моделью.

In [10]:
for col in ['vehicle_type', 'gearbox', 'fuel_type']:
    data[col].fillna(data[col].mode()[0], inplace=True)

data['model'].fillna(data['model'].value_counts().idxmax(), inplace=True)
data['repaired'].fillna('unknown', inplace=True)

data.isnull().sum()

price                 0
vehicle_type          0
registration_year     0
gearbox               0
power                 0
model                 0
kilometer             0
registration_month    0
fuel_type             0
brand                 0
repaired              0
dtype: int64

Удаление аномалий в столбце *price* и замена аномальных значений в столбце *power* на медиану.

In [11]:
power_median = data['power'].median()
power_std = data['power'].std()
power_upper_threshold = power_median + 3 * power_std
data.loc[data['power'] > power_upper_threshold, 'power'] = power_median

price_median = data['price'].median()
price_std = data['price'].std()
price_upper_threshold = price_median + 3 * price_std
data = data[data['price'] <= price_upper_threshold]

registration_year_median = data['registration_year'].median()
registration_year_std = data['registration_year'].std()
registration_year_upper_threshold = registration_year_median + 3 * registration_year_std
data = data[data['registration_year'] <= registration_year_upper_threshold]

In [12]:
data['registration_year'].sort_values(ascending=True).iloc[60:80]

218241    1800
139360    1800
195855    1800
34332     1800
351682    1800
29383     1910
302641    1910
69320     1910
87717     1910
57063     1910
321782    1910
303721    1910
334901    1910
261973    1910
218273    1910
353961    1910
78374     1910
224891    1910
193266    1910
29516     1910
Name: registration_year, dtype: int64

В качестве наименьшего возможного значения столбца *registration_year* возьмем 1910 год, в качестве максимального - 2016.

In [13]:
data = data[(data.registration_year >= 1910) & (data.registration_year <= 2016)]

Теперь обработаем нижний диапазон значений в столбце *power*. Перейдем на auto.ru, поставим максимально возможное значение Л.С равное 10. В России продается только одна такая машина - СМЗ С-3А С-3АМ. За минимальное возможное значение Л.С. возьмем 10.

In [14]:
data = data[(data.power >= 10)]

In [15]:
data['price'].sort_values(ascending=True).iloc[9500:10300]

43981     150
240962    150
91531     150
276582    150
220034    150
         ... 
42937     200
42945     200
151307    200
144117    200
70592     200
Name: price, Length: 800, dtype: int64

В качестве наименьшего возможного значения столбца *price* возьмем 200. Так стоит самая дешевая машина на auto.ru

In [16]:
data = data[(data.price >= 200)]

In [17]:
data = pd.get_dummies(data, columns=['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired'], drop_first=True)

Датасет был проанализирован, названия столбцов приведены к snake_case, были удалены аномалии, к категориальным признакам был применен *get_dummies*.

## Обучение моделей

Разделение данных на обучающую и тестовую выборки, создание функции *rmse_scorer* для ее дальнейшего использования в параметрах GridSearchCV.

In [18]:
features = data.drop('price', axis=1)
target = data['price']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

**Модель LightGBM**

In [19]:
model_lgb = lgb.LGBMRegressor()

param_grid_lgb = {
   'n_estimators': [100, 200, 300], 
   'learning_rate': [0.01, 0.05, 0.1], 
   'max_depth': [3, 5, -1]
}

grid_search_lgb = GridSearchCV(estimator=model_lgb, param_grid=param_grid_lgb, scoring='neg_root_mean_squared_error', cv=3)
grid_search_lgb.fit(features_train, target_train)
best_params_lgb = grid_search_lgb.best_params_
best_rmse_lgb = grid_search_lgb.best_score_ * -1
print("LightGBM")
print("RMSE:", best_rmse_lgb)
print("Лучшие параметры:", best_params_lgb)

LightGBM
RMSE: 1426.092474160897
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 300}


**Модель CatBoostRegressor**

In [20]:
model_cb = CatBoostRegressor(verbose=False)

param_grid_cb = {
   'learning_rate': [0.1, 0.01, 0.001],
   'depth': [4, 6, 8],
   'iterations': [100, 200, 300]
}

grid_search_cb = GridSearchCV(estimator=model_cb, param_grid=param_grid_cb, scoring='neg_root_mean_squared_error', cv=3)
grid_search_cb.fit(features_train, target_train)

best_params_cb = grid_search_cb.best_params_
best_rmse_cb = grid_search_cb.best_score_ * -1
print("CatBoostRegressor")
print("RMSE:", best_rmse_cb)
print("Лучшие параметры:", best_params_cb)

CatBoostRegressor
RMSE: 1450.1476281628009
Лучшие параметры: {'depth': 8, 'iterations': 300, 'learning_rate': 0.1}


**Модель LinearRegression**

Подготовка к масштабированию.

In [21]:
nums = ['registration_year', 'kilometer', 'power']
scaler = StandardScaler()
features_train_lr = features_train.copy()
features_test_lr = features_test.copy()
features_train_lr[nums] = scaler.fit_transform(features_train_lr[nums])
features_test_lr[nums] = scaler.transform(features_test_lr[nums])

In [22]:
model_lr = LinearRegression()

param_grid_lr = {}

grid_search_lr = GridSearchCV(model_lr, param_grid_lr, scoring='neg_root_mean_squared_error', cv=5)
grid_search_lr.fit(features_train_lr, target_train)

best_params_lr = grid_search_lr.best_params_
best_rmse_lr = grid_search_lr.best_score_ * -1
print("LinearRegression")
print("RMSE:", best_rmse_lr)
print("Лучшие параметры:", best_params_lr)

LinearRegression
RMSE: 2298.8016943493967
Лучшие параметры: {}


## Анализ моделей

Модель LightGBM показала наилучший результат RMSE - 1600.5875133724232, CatBoostRegressor - 1625.0127033919537.

LinearRegression показала неудовлетворительный результат RMSE, равный 2628.80613674923.

Рассмотрим модели LightGBM и CatBoostRegressor.

**LightGBM**

In [23]:
best_model_lgbm = lgb.LGBMRegressor(learning_rate=0.1, max_depth=-1, n_estimators=300)

start_time = time.time()
best_model_lgbm.fit(features_train, target_train)
end_time = time.time()
print("Время обучения:", end_time - start_time)

start_time = time.time()
best_model_lgbm_predicted = best_model_lgbm.predict(features_train)
end_time = time.time()
print("Время предсказания:", end_time - start_time)

Время обучения: 1.1930139064788818
Время предсказания: 0.676377534866333


**CatBoostRegressor**

In [27]:
best_model_cbr = CatBoostRegressor(depth=8, iterations=300, learning_rate=0.1, verbose=False)

start_time = time.time()
best_model_cbr.fit(features_train, target_train)
end_time = time.time()
print("Время обучения:", end_time - start_time)

start_time = time.time()
best_model_cbr_predicted = best_model_cbr.predict(features_train)
end_time = time.time()
print("Время предсказания:", end_time - start_time)

Время обучения: 3.6803102493286133
Время предсказания: 0.0578460693359375


**LinearRegression**

In [26]:
best_model_lr = LinearRegression()

start_time = time.time()
best_model_lr.fit(features_train_lr, target_train)
end_time = time.time()
print("Время обучения:", end_time - start_time)

start_time = time.time()
best_model_lr_predicted = best_model_lr.predict(features_train_lr)
end_time = time.time()
print("Время предсказания:", end_time - start_time)

Время обучения: 4.076190710067749
Время предсказания: 0.1810903549194336


Лучшей моделью выбираем LightGBM ввиду очень быстрой скорости обучения (1.13 сек) по сравнению с CatBoostRegressor (3.7 сек) и LinearRegression (4.08). LightGBM также показала отличное значение RMSE на кросс-валидации - 1412.4

**Тестирование лучшей модели**

In [31]:
start_time = time.time()
best_model_lgbm_predicted = best_model_lgbm.predict(features_test)
end_time = time.time()
print("Время предсказания лучшей модели на тестовой выборке:", end_time - start_time)
print("RMSE лучшей модели на тестовой выборке:", np.sqrt(mean_squared_error(target_test, best_model_lgbm_predicted)))

Время предсказания лучшей модели на тестовой выборке: 0.20096826553344727
RMSE лучшей модели на тестовой выборке: 1412.030024433636


## Вывод

В ходе работы был проведен анализ датасета, приведены к snake_case названия столбцов. Удалены неинформативные признаки, обработаны пропуски и аномалии. 

Были проанализрованы три модели - CatBoostRegressor, LightGBM и LinearRegression. В качестве лучшей выбрана LightGBM ввиду высокой точности (RMSE - 1412) и крайне маленьким временем обучения - 0.2 сек.